# Converting .sav to .fits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sav_manip import sav_dict
from astropy.io import fits
import datetime

`astropy.io.fits` is pretty simple to open a FITS file:

In [2]:
hd = fits.open('A000001.fits')

List the header/data units:

In [4]:
hd.info()

Filename: A000001.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1  Single Dish    1 BinTableHDU     55   1024R x 5C   [1D, 1D, 1D, 1D, 1E]   


Since this is a binary table file the first hdu is simple:

In [6]:
hd[0].header

SIMPLE  =                    T /Written by IDL:  Tue Mar 13 21:43:39 2018       
BITPIX  =                    8 /                                                
NAXIS   =                    0 /                                                
EXTEND  =                    T /File contains extensions                        
HISTORY Spectrum from ALFALFA src file 2018                                     

The second hdu has most of the metadata, and the data:

In [7]:
hd[1].header

XTENSION= 'BINTABLE'           /Written by IDL:  Tue Mar 13 21:43:39 2018       
BITPIX  =                    8 /                                                
NAXIS   =                    2 /Binary table                                    
NAXIS1  =                   36 /Number of bytes per row                         
NAXIS2  =                 1024 /Number of rows                                  
PCOUNT  =                    0 /Random parameter count                          
GCOUNT  =                    1 /Group count                                     
TFIELDS =                    5 /Number of columns                               
EXTNAME = 'Single Dish'        /ALFALFA Survey                                  
OBSERVAT= 'Arecibo Observatory' /Designation of Observatory                     
TELESCOP= 'Arecibo Radio Telescope' /Designation of Telescope                   
INSTRUME= 'ALFA L-band Feed Array' /Instrument in use                           
BEAM    = '3.3x3.8 '        

The astropy fits routines bring in useful methods for examining HDUs. You cand address the table columns with keynames:

In [6]:
hd[1].data['VHELIO']

array([7956.75513663, 7951.32521979, 7945.89549456, ..., 2510.81970251,
       2505.58025946, 2500.34099803])

The following lists the columns of the bintable. It is the UAT standard to put spectral data as column vectors, not as row vectors within a column.

In [7]:
hd[1].columns

ColDefs(
    name = 'VHELIO'; format = '1D'; unit = 'km/s'
    name = 'FREQUENCY'; format = '1D'; unit = 'MHz'
    name = 'FLUX'; format = '1D'; unit = 'mJy'
    name = 'BASELINE'; format = '1D'; unit = 'mJy'
    name = 'WEIGHT'; format = '1E'
)

## Data conversion

The `sav_manip.sav_dict` function opens the LBW data structure from IDL and puts the elements in a python dictionary. Most of its elements are reduced to scalars or simple arrays keyed by their keywords. A few are more nested structures that need a little more manipulation to be placed in the FITS HDUs.

It unpacks the `lbwsrc` variable in the idl structure.

In [33]:
# get some APPSS data:
appssd = sav_dict('S000018.4+273720.sav')

--------------------------------------------------
Date: Mon Dec  3 12:43:54 2018
User: a2010
Host: galfas1.naic.edu
--------------------------------------------------
Format: 9
Architecture: x86_64
Operating System: linux
IDL Version: 7.1
--------------------------------------------------
Successfully read 5 records of which:
 - 1 are of type VERSION
 - 1 are of type TIMESTAMP
 - 1 are of type NOTICE
 - 1 are of type VARIABLE
--------------------------------------------------
Available variables:
 - lbwsrc [<class 'numpy.recarray'>]
--------------------------------------------------


In [34]:
for thing in appssd.items():
    print(thing)

('AGC', 0)
('LBWSRCNAME', b'S000018.4+273720')
('CORFILE', '')
('FILE', b'S000018.4+273720')
('HIRES', 0)
('RA', 0.07666686706532444)
('DEC', 27.622222409049655)
('BOARD', 3)
('SPEC', array([ 0.96925426,  0.24481145,  0.04342056, ...,  0.61296   ,
        0.16968232, -0.10034379], dtype=float32))
('RAW', array([-1.7264147, -3.0506778, -3.304659 , ..., -2.0318284, -2.575946 ,
       -2.845447 ], dtype=float32))
('FREQ', array([1362.8078669 , 1362.82007393, 1362.83228096, ..., 1387.77124581,
       1387.78345284, 1387.79565987]))
('VEL', array([12648.92478552, 12646.12619031, 12643.32764524, ...,
        7028.6954804 ,  7025.99666208,  7023.29789124]))
('NCHAN', 2048)
('BANDWIDTH', 24.98779296875)
('MASK', array([0, 0, 0, ..., 0, 0, 0], dtype=int16))
('BASELINE', array([-3.02633962, -3.02791991, -3.02949547, ..., -2.67597079,
       -2.67697328, -2.67797962]))
('BLFIT', rec.array([(1, 3, array([-3.00770900e+01,  9.08550641e-03, -9.74057762e-07,  3.35875993e-11,
        0.00000000e+00,  0

### Assemble parts of FITS file

Now make the astropy data structure for a new fits file.

In [78]:
tstamp = datetime.datetime.utcnow()
t_str = tstamp.isoformat() + ' UTC'
#This makes the first (primary) HDU, which is not very complicated.
hdr = fits.Header()
hdr['COMMENT'] = 'TEST .FITS file from idl .sav made using python'
hdr['HISTORY'] = 'David Craig for APPSS'
hdr['HISTORY'] = 'Generated ' + t_str
primary_hdu = fits.PrimaryHDU(header=hdr)


In [79]:
t_str # check the format of the time string

'2019-08-10T16:02:10.990036 UTC'

In [80]:
# This makes the binary table hdu:
col1 = fits.Column(name='VHELIO', format='D', array=appssd['VEL'], coord_unit='km/s')
col2 = fits.Column(name='FREQUENCY', format='D', array=appssd['FREQ'], coord_unit='MHz')
col3 = fits.Column(name='FLUX', format='D', array=appssd['SPEC'], coord_unit='mJy')
col4 = fits.Column(name='BASELINE', format='D', array=appssd['BASELINE'], coord_unit='mJy')
cols = fits.ColDefs([col1,col2,col3,col4])
hdu = fits.BinTableHDU.from_columns(cols)

In [81]:
# Now put in various keywords
hdr = hdu.header
     
hdr.set('EXTNAME', 'Single Dish', 'APPSS Survey')
hdr.set('OBSERVAT', 'Arecibo Observatory','Designation of Observatory')
hdr.set('TELESCOP','Arecibo Radio Telescope','Designation of Telescope')
hdr.set('INSTRUME','LBW receiver','Instrument in use')
hdr.set('BEAM','3.3x3.8 ', 'Beam size [arcminutes] REVISE FOR LBW?')  #TODO                      
hdr.set('NANVALUE',-999.000,'Value of missing/null data')                     
hdr.set('OBJECT' , 'UNASSIGNED  ', 'Name of observed object')  #TODO NEEDS IMPORT if found               
hdr.set( 'NAME'    , ' ' ,'Common name' )   #TODO NEEDS IMPORT if found 
#appss idl text data is still in bytes, does it work? NOPE! So it must be decoded:                      
hdr.set('HISRC ', appssd['LBWSRCNAME'].decode('ascii'), 'HI source name')    
hdr.set('ORIGIN', 'WTAMU via SciServer', 'File creation location') #TODO: software id instead?
hdr.set('RA', appssd['RA'],'Right ascension in degrees')
hdr.set('DEC', appssd['DEC'], 'Declination in degrees')
hdr.set('EPOCH',2000.0,'Epoch for coordinates (years)')
hdr.set('CHAN', appssd['NCHAN'], 'Number of spectral channels')
hdr.set('BW', appssd['BANDWIDTH'],'Bandwidth [MHz]')


The following are APPSS style data, which are a little different from what I see in Martha's reduced ALFALFA FITS files:
  
     'MASK',     # mask doesnt appear in the ALFALFA files, should it here?
     'BASELINE', # baseline doesnt appear in the ALFALFA files, should it here?
     'BLFIT',    # and the polynomial?
     'CONTINUUM',
     'WINDOW',
     'RMS',
     'FITTYPE',
     'FITEDGE',
     'W50',
     'W50ERR',
     'W20',
     'W20ERR',
     'VSYS',
     'VSYSERR',
     'FLUX',
     'FLUXERR',
     'COMMENTS',
     'SN',

In [82]:
hdr.set('WINDOW', appssd['WINDOW'].decode('ascii'), 'h for Hamming(?)')
hdr.set('FITTYPE', appssd['FITTYPE'].decode('ascii'), 'P for polynomial')
hdr.set('RMS', appssd['RMS'], 'RMS of spectrum (noise)')
hdr.set('W50', appssd['W50'], 'estimated velocity width at 50%')
hdr.set('W50ERR', appssd['W50ERR'], 'error in W50')
hdr.set('W20', appssd['W20'], 'estimated velocity width at 20%')
hdr.set('W20ERR', appssd['W20ERR'], 'error in W20')
hdr.set('VSYS', appssd['VSYS'], 'systemic velocity [km/s]')
hdr.set('VSYSERR',appssd['VSYSERR'], 'error in Vsys [km/s]')
hdr.set('FLUX', appssd['FLUX'], 'integrated flux [Jy km/s]')
hdr.set('FLUXERR', appssd['FLUXERR'], 'error in integrated flux')
hdr.set('SN',appssd['SN'],'estimated signal-to-noise ratio')

In [83]:
# now all comments from the sav structure that are in the dict
# this will insert a new comment for each of those lines.
for line in range(appssd['COMMENTS'].COUNT[0]):
     hdr.set('COMMENT',appssd['COMMENTS'].TEXT[0][line].decode('ascii'))

In [84]:
#comments for test file
hdr.set('COMMENT', 'WARNING! This is a test file from development software!')

In [85]:
hdu_lst = fits.HDUList([primary_hdu, hdu])
hdu_lst.writeto('APPSS_test_tab.fits', overwrite = True)

### Read the file back for examination

In [86]:
hdtest = fits.open('APPSS_test_tab.fits')

In [87]:
hdtest[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT TEST .FITS file from idl .sav made using python                         
HISTORY David Craig for APPSS                                                   
HISTORY Generated 2019-08-10T16:02:10.990036 UTC                                

In [88]:
hdtest[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   32 / length of dimension 1                          
NAXIS2  =                 2048 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    4 / number of table fields                         
TCUNI1  = 'km/s    '                                                            
TCUNI2  = 'MHz     '                                                            
TCUNI3  = 'mJy     '                                                            
TCUNI4  = 'mJy     '                                                            
TTYPE1  = 'VHELIO  '        